In [47]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
import sklearn
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

In [48]:
language = 'fsl'

In [49]:
datasets = {chr(letter):pd.read_csv(f'processed_data/{language}_mp01021/{chr(letter)}_Data.csv') for letter in range(ord('A'), ord('Z') + 1)}

In [50]:
datasets['A'].head()

,Landmarks,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,...,17.2,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2
0,Coordinates,x,y,z,x,y,z,x,y,z,...,z,x,y,z,x,y,z,x,y,z
1,Frame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,-0.328798770904541,0.9440675973892212,-0.025055259466171265,0.09019581228494644,0.9056023955345154,-0.2417997121810913,0.46820488572120667,0.5142858624458313,-0.30139783024787903,...,-0.08161065727472305,-0.49492689967155457,-0.06241708993911743,-0.1844290941953659,-0.4510268270969391,0.20504318177700043,-0.10821167379617691,-0.46734902262687683,0.2073359340429306,-0.024087782949209213
3,2,-0.2267197072505951,0.9729245901107788,-0.044898826628923416,0.20533159375190735,0.8290771842002869,-0.22340598702430725,0.5266153812408447,0.4613969922065735,-0.29489830136299133,...,-0.09364113956689835,-0.4352213144302368,-0.06646435707807541,-0.1759088933467865,-0.39210861921310425,0.18160800635814667,-0.09695502370595932,-0.4101670980453491,0.21136802434921265,-0.012231195345520973
4,3,-0.29781967401504517,0.9545851945877075,-0.008398245088756084,0.12096328288316727,0.8573551177978516,-0.1993054896593094,0.4726623594760895,0.46044933795928955,-0.22897109389305115,...,-0.008828305639326572,-0.4492706060409546,-0.06148320436477661,-0.11127342283725739,-0.42576828598976135,0.19160698354244232,-0.04531983286142349,-0.4427775740623474,0.2054169476032257,0.03030148148536682


In [51]:
for key, value in datasets.items():
    datasets[key] = datasets[key].drop([0,1])
    datasets[key] = datasets[key].drop(columns=['Landmarks'])
    datasets[key]['Letter'] = key
    datasets[key] = datasets[key].sample(frac=1).reset_index(drop=True)

In [52]:
datasets['B'].head()

,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,3,...,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2,Letter
0,-0.2160150557756424,0.9738113880157471,0.07091488689184189,0.0934617817401886,0.815496027469635,-0.1321834772825241,0.28832191228866577,0.4936845600605011,-0.2058582305908203,0.18041805922985077,...,-0.3152645230293274,-0.23754025995731354,-0.08563493192195892,-0.2894911766052246,-0.4183290898799896,-0.10719995945692062,-0.27550962567329407,-0.5794624090194702,-0.12912197411060333,B
1,0.04733399674296379,0.9986663460731506,-0.020615147426724434,0.2659691870212555,0.708798885345459,-0.07734376192092896,0.2931068539619446,0.3462115228176117,-0.10604765266180038,0.15034213662147522,...,-0.3579603135585785,-0.09448505938053131,-0.15175677835941315,-0.36523526906967163,-0.2766677439212799,-0.1512681394815445,-0.3506331443786621,-0.4345927834510803,-0.16070885956287384,B
2,-0.23535026609897614,0.9714571833610535,0.029685866087675095,0.06982066482305527,0.7957343459129333,-0.12185420840978622,0.24160784482955933,0.47368091344833374,-0.17810672521591187,0.12149278074502945,...,-0.3361363112926483,-0.20175395905971527,-0.12497439980506897,-0.30979472398757935,-0.39697861671447754,-0.16557501256465912,-0.2781464457511902,-0.5613037943840027,-0.20407575368881226,B
3,-0.28253841400146484,0.9558480381965637,0.08078692108392715,0.012358074076473713,0.8269078731536865,-0.07525411993265152,0.20931784808635712,0.5194309949874878,-0.14531759917736053,0.09083038568496704,...,-0.3459891974925995,-0.23487721383571625,-0.11537469178438187,-0.32579267024993896,-0.42646944522857666,-0.14217709004878998,-0.3046497106552124,-0.5919105410575867,-0.17248117923736572,B
4,-0.21660427749156952,0.972685694694519,0.08345707505941391,0.08000756800174713,0.8180012702941895,-0.10257203876972198,0.2874310612678528,0.5286237597465515,-0.1805153489112854,0.20014503598213196,...,-0.3202137351036072,-0.24163660407066345,-0.1002151146531105,-0.29706698656082153,-0.4221753478050232,-0.10861340165138245,-0.2812380790710449,-0.5861000418663025,-0.11843950301408768,B


In [53]:
data = pd.concat(datasets.values(), ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,3,...,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2,Letter
0,0.21171773970127106,0.9767455458641052,0.03381861373782158,0.3350772559642792,0.6495612263679504,-0.07947279512882233,0.24096792936325073,0.28733158111572266,-0.08771830797195435,-0.034243084490299225,...,-0.2779718339443207,0.0723707303404808,-0.20700660347938538,-0.19039306044578552,0.14431586861610413,-0.2636958360671997,-0.10366441309452057,0.27821317315101624,-0.3136678636074066,W
1,0.23789885640144348,0.9710900187492371,-0.01970669813454151,0.32325446605682373,0.5948726534843445,-0.04934968426823616,0.2877626419067383,0.19636133313179016,-0.0561339370906353,0.12921178340911865,...,-0.39787834882736206,0.16204093396663666,-0.2172173708677292,-0.3221866190433502,0.273086816072464,-0.18090973794460297,-0.22792205214500427,0.35473859310150146,-0.13388493657112122,Z
2,-0.2009689211845398,0.751974880695343,0.6278098821640015,0.06599192321300507,0.33265408873558044,0.5956430435180664,0.2725592851638794,-0.11229388415813446,0.45672324299812317,0.3784763216972351,...,0.5222320556640625,0.613762617111206,-0.21370740234851837,0.7651430368423462,0.582495927810669,-0.18964943289756775,0.9263443350791931,0.530733585357666,-0.17455649375915527,J
3,-0.20091217756271362,0.9792382121086121,0.026960350573062897,0.11952028423547745,0.8078165650367737,-0.11175139248371124,0.41159653663635254,0.5200336575508118,-0.17326399683952332,0.6729715466499329,...,-0.4504903554916382,-0.17027460038661957,-0.196145698428154,-0.3891325294971466,-0.05360081046819687,-0.17392246425151825,-0.3532902002334595,0.07920940220355988,-0.1340995728969574,L
4,-0.48754602670669556,0.8718947172164917,0.04580916464328766,-0.12789186835289001,0.8026905655860901,-0.047212205827236176,0.21725067496299744,0.49975505471229553,-0.08415017277002335,0.19894394278526306,...,-0.41272974014282227,-0.3878060281276703,-0.1659405678510666,-0.3909347355365753,-0.5976364016532898,-0.16672684252262115,-0.34657663106918335,-0.8242152333259583,-0.14773975312709808,I


In [54]:
y = data['Letter']
X = data.drop(columns=['Letter'])

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.3, random_state=42)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_train = X_train.dropna()
X_train = X_train.to_numpy()
X_val = X_val.apply(pd.to_numeric, errors='coerce')
X_val = X_val.dropna()
X_val = X_val.to_numpy()
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = X_test.dropna()
X_test = X_test.to_numpy()


le = LabelEncoder()
y_train = le.fit_transform(y_train)  
y_val = le.transform(y_val)  
y_test = le.transform(y_test)

In [56]:
X_train.shape

(6502, 63)

In [57]:
# Define model function
def create_keras_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),  # Helps prevent overfitting

        keras.layers.Dense(64, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(32, activation='relu'),

        keras.layers.Dense(26, activation='softmax')  # 26 classes
    ])

    # Compile the model
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',  # Use if labels are integers (0-25)
                metrics=['accuracy'])
    return model


In [58]:
keras_model = create_keras_model()
keras_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_val, y_val))

d:\Fingerspelling-Recognition\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
val_loss, val_acc = keras_model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {val_acc:.4f}")

Final Validation Accuracy: 0.9921


In [60]:
test_loss, test_acc = keras_model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Accuracy: {test_acc:.4f}")

Final Test Accuracy: 0.9875


In [61]:
keras_model.save(f"models/{language}/keras_ANN.keras")

In [62]:
def create_xgb_model():
    model = xgb.XGBClassifier(
        objective="multi:softmax",  # Multi-class classification
        num_class=26,               # 26 different classes
        eval_metric="mlogloss",      # Multiclass log loss
        max_depth=6,                 # Depth of trees (adjust based on experiments)
        learning_rate=0.1,           # Step size shrinkage (adjust for tuning)
        n_estimators=100,            # Number of trees (increase for better results)
        use_label_encoder=False      # Required for newer versions of XGBoost
    )
    return model

In [63]:
xgb_model = create_xgb_model()
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

d:\Fingerspelling-Recognition\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:00:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mlogloss:2.29116
[1]	validation_0-mlogloss:1.91630
[2]	validation_0-mlogloss:1.66046
[3]	validation_0-mlogloss:1.46657
[4]	validation_0-mlogloss:1.30844
[5]	validation_0-mlogloss:1.17836
[6]	validation_0-mlogloss:1.06737
[7]	validation_0-mlogloss:0.97171
[8]	validation_0-mlogloss:0.88723
[9]	validation_0-mlogloss:0.81215
[10]	validation_0-mlogloss:0.74622
[11]	validation_0-mlogloss:0.68657
[12]	validation_0-mlogloss:0.63387
[13]	validation_0-mlogloss:0.58552
[14]	validation_0-mlogloss:0.54217
[15]	validation_0-mlogloss:0.50267
[16]	validation_0-mlogloss:0.46628
[17]	validation_0-mlogloss:0.43319
[18]	validation_0-mlogloss:0.40361
[19]	validation_0-mlogloss:0.37625
[20]	validation_0-mlogloss:0.35135
[21]	validation_0-mlogloss:0.32841
[22]	validation_0-mlogloss:0.30761
[23]	validation_0-mlogloss:0.28842
[24]	validation_0-mlogloss:0.27112
[25]	validation_0-mlogloss:0.25515
[26]	validation_0-mlogloss:0.24053
[27]	validation_0-mlogloss:0.22698
[28]	validation_0-mlogloss:0.2

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=26, num_parallel_tree=None, ...)

In [64]:
# Make predictions
y_pred_val = xgb_model.predict(X_val)

# Compute accuracy
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Evaluation Accuracy: {accuracy:.4f}")

Evaluation Accuracy: 0.9885


In [65]:
# Make predictions
y_pred_test = xgb_model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9828


In [66]:
xgb_model.save_model(f"models/{language}/xgboost_model.json")

In [67]:
X_test

array([[ 0.07117779,  0.99634153,  0.04730054, ..., -0.23675866,
         0.39642996, -0.11813724],
       [-0.2177745 ,  0.97414345,  0.06015664, ..., -0.35216853,
         0.09757519, -0.07609792],
       [-0.14845969,  0.98777109,  0.04762341, ...,  0.68911678,
         0.10101629, -0.14755474],
       ...,
       [-0.3642002 ,  0.92572355, -0.10195162, ..., -0.45715696,
         0.17325392, -0.04867852],
       [-0.09692637,  0.99232531,  0.07678401, ..., -0.12298288,
         0.52345651, -0.13003486],
       [-0.1798263 ,  0.97497064,  0.13074669, ..., -0.48591673,
        -0.05828797, -0.12163033]])

In [68]:
X_test = X_test[0].reshape(1, -1)
X_test

array([[ 0.07117779,  0.99634153,  0.04730054,  0.27384338,  0.72468346,
        -0.11597569,  0.2595152 ,  0.37949711, -0.1675832 , -0.0186281 ,
         0.16900374, -0.21901874, -0.27504542,  0.0476688 , -0.25372189,
         0.23688878, -0.02123114,  0.00586876,  0.29533234, -0.43121073,
        -0.10775971,  0.31384465, -0.69979954, -0.17469797,  0.32242078,
        -0.92468435, -0.20549747,  0.        ,  0.        ,  0.        ,
        -0.10455015, -0.46015909, -0.12994802, -0.19490489, -0.73943728,
        -0.19671227, -0.27647993, -0.95855331, -0.20890903, -0.18620028,
         0.12179077, -0.03532141, -0.29292914, -0.04322465, -0.25835791,
        -0.18804888,  0.17595395, -0.28775382, -0.0998941 ,  0.33021483,
        -0.22406849, -0.33081546,  0.29185092, -0.08118614, -0.43182147,
         0.1150532 , -0.22437821, -0.33023369,  0.26445407, -0.19043458,
        -0.23675866,  0.39642996, -0.11813724]])

In [69]:
t = xgb_model.predict(X_test[0].reshape(1, -1))

In [70]:
t[0]

21